# Dataset ingestion

This jupyter noteebook ingests the [Geocoded National Address File][gnaf] ([GNAF][gnaf]) from [data.gov.au](data.gov.au). It also downloads the [land values for NSW][nswlv], and ABS shapefiles 

It loads it all this data into a PostgreSQL database in a docker container, treating it like a disposable sqlite data store. It also downloads the ABS shape files as well as the 

Here we are going to ingest all the data necessary in order to assess land by land values, and filter them by address information. 

### The Steps

1. Download static assets and datasets
2. Setup a docker container with postgresql with GIS capabilities.
3. Ingest the [ABS shape files][abssf]
4. Ingest the latest [NSW valuer general land values][nswlv].
5. Ingest the [Geocoded National Address File][gnaf] ([GNAF][gnaf]) dataset
6. Link NSW Valuer General data with GNAF dataset

[gnaf]: https://data.gov.au/data/dataset/geocoded-national-address-file-g-naf
[nswlv]: https://www.valuergeneral.nsw.gov.au/land_value_summaries/lv.php
[abssf]: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

### Note

- Make sure docker is running first.

### Warning

Do not connect this to another database unless you've taken the time to update this, as it'll drop the existing database. I suggest instead take what you need from this script and disregard the rest. DO NOT USE DATABASE CREDENTIALS HERE FOR ANY OTHER STORE (especailly anything with drop permissions).

It also executes sql from a zip file downloaded from an external source.


## Configuration

These are some fields to configure if you wish to configure how the data is injected.

In [1]:
from lib import notebook_constants as nc

GLOBAL_FLAGS = {
    # If you mark this as true, the table `nsw_valuer_general.raw_entries`
    # will be dropped. If you have space limitations and no desire to debug
    # the data than dropping this makes sense. If you wish to debug some values
    # then keeping this around may make some sense.
    'drop_raw_nsw_valuer_general_entries': True,
    'reinitialise_container': True,
}

db_conf = nc.gnaf_dbconf
db_name = nc.gnaf_dbname

docker_container_name = 'gnaf_db_prod'
docker_image_tag = "20240908_19_53"

## Download Static Files

Here we are downloading static files, as well as fetching the most recently published land values from the valuer generals website.

In [2]:
import logging
from lib.service.io import IoService
from lib.tasks.fetch_static_files import initialise, get_session

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

io_service = IoService.create(None)
async with get_session(io_service) as session:
    environment = await initialise(io_service, session)

land_value_dis = environment.land_value
w_sale_price = environment.sale_price_weekly
a_sale_price = environment.sale_price_annual
gnaf_dis = environment.gnaf

2024-10-01 19:34:17,521 - INFO - Checking Target "abs_main_structures.zip"
2024-10-01 19:34:17,521 - INFO - Checking Target "non_abs_shape.zip"
2024-10-01 19:34:17,522 - INFO - Checking Target "g-naf_aug24_allstates_gda2020_psv_1016.zip"
2024-10-01 19:34:17,522 - INFO - Checking Target "nswvg_lv_01_Oct_2024.zip"
2024-10-01 19:34:17,523 - INFO - Checking Target "nswvg_wps_01_Jan_2024.zip"
2024-10-01 19:34:17,523 - INFO - Checking Target "nswvg_wps_08_Jan_2024.zip"
2024-10-01 19:34:17,523 - INFO - Checking Target "nswvg_wps_15_Jan_2024.zip"
2024-10-01 19:34:17,524 - INFO - Checking Target "nswvg_wps_22_Jan_2024.zip"
2024-10-01 19:34:17,524 - INFO - Checking Target "nswvg_wps_29_Jan_2024.zip"
2024-10-01 19:34:17,525 - INFO - Checking Target "nswvg_wps_05_Feb_2024.zip"
2024-10-01 19:34:17,525 - INFO - Checking Target "nswvg_wps_12_Feb_2024.zip"
2024-10-01 19:34:17,526 - INFO - Checking Target "nswvg_wps_19_Feb_2024.zip"
2024-10-01 19:34:17,526 - INFO - Checking Target "nswvg_wps_26_Feb_202

## Create Container with Database

Here we are creating a container in docker from an image that uses the postgres image, which also installs a few extensions.

### Note

This notebook this is designed to be run more than once, so it'll throw away any existing container and database before creating a new one. After getting rid of any container using the same identifer, it'll create a new one and pull the relevant image if it's not already installed. It'll wait till the postgres instance is live then create the database. 

In [3]:
from lib.gnaf_db import GnafDb, GnafContainer, GnafImage
from lib import notebook_constants as nc

if GLOBAL_FLAGS['reinitialise_container']:
    image = GnafImage.create(tag=docker_image_tag)
    image.prepare()
    
    container = GnafContainer.create(container_name=docker_container_name, image=image)
    container.clean()
    container.prepare(db_conf, db_name)
    container.start()
else:
    print('skipping container initialisation')

gnaf_db = GnafDb.create(db_conf, db_name)
gnaf_db.wait_till_running()

if GLOBAL_FLAGS['reinitialise_container']:
    gnaf_db.init_schema(gnaf_dis.publication)
else:
    print('skipping DB initialisation')
    raise Exception()

running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/create_tables_ansi.sql
running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/add_fk_constraints.sql
running sql/move_gnaf_to_schema.sql


## Consume the ABS Shapefiles

The [ABS provides a number of shape files][all abs shape files], we're going focus on 2 main sets of shapes. The **ABS Main Structures** which is stuff like SA1, 2, 3 & 4 along with greater cities, meshblocks, and states. As well as **Non ABS Main Structures** which is stuff like electoral divisions, suburbs post codes etc.

[all abs shape files]: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

### ABS Main Structures 

Any address or region we look up in the GNAF dataset, we want to visualise. The ABS has a few different geographic groups which we can visualise the data against, but each address in the GNAF dataset has a meshblock id, which is the smaller block the ABS breaks addresses up into for SA1, SA2, SA3 and SA4's.

This dataset is pretty useful for visualising the GNAF data for that reason.

### Non Abs Main Structures 

We are mostly ingesting these to make it simpler to narrow data of interest. Typically if you're looking at this data, you're probably doing it some scope of relevance, such as a local government area, an electorate division, or whatever.

In [4]:
from lib.tasks.ingest_abs import ingest

await ingest(gnaf_db)

Populated abs_main_structures.state with 10/10 rows.
Populated abs_main_structures.gccsa with 35/35 rows.
Populated abs_main_structures.sa4 with 108/108 rows.
Populated abs_main_structures.sa3 with 359/359 rows.
Populated abs_main_structures.sa2 with 2473/2473 rows.
Populated abs_main_structures.sa1 with 61845/61845 rows.
Populated abs_main_structures.meshblock with 368286/368286 rows.
Populated non_abs_main_structures.localities with 15353/15353 rows.
Populated non_abs_main_structures.state_electoral_division_2021 with 452/452 rows.
Populated non_abs_main_structures.state_electoral_division_2022 with 452/452 rows.
Populated non_abs_main_structures.state_electoral_division_2024 with 452/452 rows.
Populated non_abs_main_structures.federal_electoral_division_2021 with 170/170 rows.
Populated non_abs_main_structures.lga_2021 with 566/566 rows.
Populated non_abs_main_structures.lga_2022 with 566/566 rows.
Populated non_abs_main_structures.lga_2023 with 566/566 rows.
Populated non_abs_main_

## Ingesting NSW Land Values

First lets just get the CSV's into the database, then we'll break it up into seperates tables, then we'll form links with the GNAF dataset.

#### Documentation on this dataset

The valuer general website has a link to documentation on interpretting that data on [this page](https://www.nsw.gov.au/housing-and-construction/land-values-nsw/resource-library/land-value-information-user-guide). I didn't link to the PDF directly as it occasionally updated and a direct link is at risk of going stale.

It's useful getting the meaning behind the codes and terms used in the bulk data.

### Steps

1. **Build the `nsw_valuer_general.raw_entries_lv` table**: Here we are
   just loading the each file from the latest land value publication
   with minimal changes, and a bit of sanitisizing.
2. **Break CSV data into sepreate relations**, Just to break up the data
   into more efficent representations of the data, and data that will be
   easier to query, we're going to perform a series of queries against
   the GNAF data before using it populate the tables we care about.
3. **Parse contents of the property description**, The `property_description`
   from the original valuer general data constains alot of information. The
   most important of which is the land parcel or `lot/plan` information.
   There is other information in there as well.
4. If specified drop the raw entries consumed in step (tho the default is
   to do exactly that. 


In [8]:
from lib.tasks.nsw_vg.init_db_schema import initialise_nsw_vg_schema, InitialiseNswVgSchemaConfig
from lib.tasks.nsw_vg.ingest_land_values import ingest_land_values, NswVgLandValueIngestionConfig

await initialise_nsw_vg_schema(
    InitialiseNswVgSchemaConfig(apply=True),
    gnaf_db,
    io_service,
)
await ingest_land_values(
    NswVgLandValueIngestionConfig(
        keep_raw=not GLOBAL_FLAGS['drop_raw_nsw_valuer_general_entries'],
    ),
    gnaf_db,
    environment.land_value.latest,
)

2024-10-01 22:27:52,413 - INFO - Step 1: Ingest raw files
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, enco

### Break CSV data into sepreate relations

Just to break up the data into more efficent representations of the data, and data that will be easier to query, we're going to perform a series of queries against the GNAF data before using it populate the tables we care about.

In [6]:
from datetime import datetime
import os
import math
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from sqlalchemy import text
from psycopg2.errors import StringDataRightTruncation

with gnaf_db.connect() as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.source_file CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.source CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.district CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.suburb CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.street CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.property CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.property_description CASCADE")
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.valuations CASCADE")
    
    with open('sql/nsw_lv_schema_2_structure.sql', 'r') as f:
        cursor.execute(f.read())
        
    with open('sql/nsw_lv_from_raw.sql', 'r') as f:
        cursor.execute(f.read())
        
    cursor.close()
    
count('district')
count('suburb')
count('street')
count('property')
count('property_description')
count('valuations')

NameError: name 'count' is not defined

### Parse contents of the property description

The `property_description` from the original valuer general data constains alot of information. The most important of which is the land parcel or `lot/plan` information. There is other information in there as well.

In [ ]:
import numpy as np
import pandas as pd
from lib.nsw_vg.property_description import parse_land_parcel_ids

engine = gnaf_db.engine()

with gnaf_db.connect() as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.land_parcel_link")
    with open('sql/nsw_lv_schema_3_property_description_meta_data.sql', 'r') as f:
        cursor.execute(f.read())
    cursor.close()

def land_parcels(desc):
    desc, parcels = parse_land_parcel_ids(desc)
    return parcels 

query = "SELECT * FROM nsw_valuer_general.property_description"
for df_chunk in pd.read_sql(query, engine, chunksize=10000):
    df_chunk = df_chunk.dropna(subset=['property_description'])
    df_chunk['parcels'] = df_chunk['property_description'].apply(land_parcels)
    df_chunk_ex = df_chunk.explode('parcels')
    df_chunk_ex = df_chunk_ex.dropna(subset=['parcels'])
    df_chunk_ex['land_parcel_id'] = df_chunk_ex['parcels'].apply(lambda p: p.id)
    df_chunk_ex['part'] = df_chunk_ex['parcels'].apply(lambda p: p.part)
    df_chunk_ex = df_chunk_ex.drop(columns=['property_description', 'parcels'])
    df_chunk_ex.to_sql(
        'land_parcel_link',
        con=engine,
        schema='nsw_valuer_general',
        if_exists='append',
        index=False,
    )

with gnaf_db.connect() as conn:
    cursor = conn.cursor()
    for t in ['property', 'land_parcel_link']:
        cursor.execute(f'SELECT COUNT(*) FROM nsw_valuer_general.{t}')
        count = cursor.fetchone()[0]
        print(f"Table nsw_valuer_general.{t} has {count} rows")


### Get rid of `raw_entries` table

We no longer need the raw entries table, deleting it should make the database a bit efficent in terms of storage.

In [ ]:
with gnaf_db.connect() as conn:
    cursor = conn.cursor()
    if GLOBAL_FLAGS['drop_raw_nsw_valuer_general_entries']:
        cursor.execute("DROP TABLE IF EXISTS nsw_valuer_general.raw_entries_lv")
        print("Dropping raw entries table")
    else:
        print("Keeping raw entries table")
    cursor.close()

## Gnaf Ingestion

Here we ingest the GNAF dataset, this will take awhile.

In [ ]:
from lib.gnaf.ingestion import ingest
ingest(gnaf_dis.publication, gnaf_db)

## Done

We've now built up the dataset, lets analysis what we got and show the contents of the database.

In [ ]:
with gnaf_db.connect() as conn:
    cursor = conn.cursor()

    for schema in [
        'nsw_valuer_general',
        'gnaf',
        'abs_main_structures',
        'non_abs_main_structures',
    ]:
        # Get the list of all tables
        cursor.execute(f"""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = '{schema}'
        """)
        tables = cursor.fetchall()
    
        # Get row count for each table
        for table in tables:
            cursor.execute(f'SELECT COUNT(*) FROM {schema}.{table[0]}')
            count = cursor.fetchone()[0]
            print(f"Table {schema}.{table[0]} has {count} rows")
    
    cursor.close()


## Todo

Include NSW Planning data ([source 1][nswps1]) ([source 2][nswps2]) ([source 3][nswps3], examples [here][dom-example] discussion [here][planning-discussion])

[nswps1]: https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer
[nswps2]: https://mapprod3.environment.nsw.gov.au/arcgis/rest/services/Planning
[nswps3]: https://www.planningportal.nsw.gov.au/opendata/dataset/online-da-data-api
[dom-example]: https://github.com/Dominic-Behrens/nsw_da_api
[planning-discussion]: https://discord.com/channels/1099200773772034066/1099200773772034070/1270743511255220366